import statements and data you need

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline


In [27]:
# dataframe for first csv

# Path to the data - select the path that works for you 
file_path = '../data-mining-semester-project/data/Census race.csv'

# Load the data into a DataFrame 
dfCensus = pd.read_csv(file_path)

dfCensus = dfCensus.dropna()
dfCensus= dfCensus.replace('!!',' ', regex=True)
dfCensus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3222 entries, 0 to 3221
Data columns (total 73 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GEO_ID   3222 non-null   object
 1   NAME     3222 non-null   object
 2   P1_001N  3222 non-null   object
 3   P1_002N  3222 non-null   object
 4   P1_003N  3222 non-null   object
 5   P1_004N  3222 non-null   object
 6   P1_005N  3222 non-null   object
 7   P1_006N  3222 non-null   object
 8   P1_007N  3222 non-null   object
 9   P1_008N  3222 non-null   object
 10  P1_009N  3222 non-null   object
 11  P1_010N  3222 non-null   object
 12  P1_011N  3222 non-null   object
 13  P1_012N  3222 non-null   object
 14  P1_013N  3222 non-null   object
 15  P1_014N  3222 non-null   object
 16  P1_015N  3222 non-null   object
 17  P1_016N  3222 non-null   object
 18  P1_017N  3222 non-null   object
 19  P1_018N  3222 non-null   object
 20  P1_019N  3222 non-null   object
 21  P1_020N  3222 non-null   object
 22  

In [26]:
# dataframe for second csv file 
file_path2 = '../data-mining-semester-project/data/Unemployment.csv'

# Load the data into a DataFrame 
dfUnemployment = pd.read_csv(file_path2)
dfUnemployment = dfUnemployment.dropna()

dfUnemployment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3129 entries, 2 to 3195
Data columns (total 92 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPS_Code                                  3129 non-null   int64  
 1   State                                      3129 non-null   object 
 2   Area_name                                  3129 non-null   object 
 3   Rural_urban_continuum_code_2013            3129 non-null   float64
 4   Urban_influence_code_2013                  3129 non-null   float64
 5   Metro_2013                                 3129 non-null   float64
 6   Civilian_labor_force_2000                  3129 non-null   object 
 7   Employed_2000                              3129 non-null   object 
 8   Unemployed_2000                            3129 non-null   object 
 9   Unemployment_rate_2000                     3129 non-null   float64
 10  Civilian_labor_force_200

In [28]:
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path3 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts.csv'

# Load the data into a DataFrame 
dfODeath = pd.read_csv(file_path3)
dfODeath = dfODeath.dropna()

dfODeath.head()


,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
10,AK,2015,April,12 month-ending,Number of Drug Overdose Deaths,126,100,0.0,Alaska,Numbers may differ from published reports usin...,**,126
12,AK,2015,August,12 month-ending,Number of Drug Overdose Deaths,124,100,0.0,Alaska,Numbers may differ from published reports usin...,**,124
30,AK,2015,December,12 month-ending,Number of Drug Overdose Deaths,121,100,0.0,Alaska,Numbers may differ from published reports usin...,**,121
38,AK,2015,February,12 month-ending,Number of Drug Overdose Deaths,127,100,0.0,Alaska,Numbers may differ from published reports usin...,**,127
49,AK,2015,January,12 month-ending,Number of Drug Overdose Deaths,126,100,0.0,Alaska,Numbers may differ from published reports usin...,**,126


In [25]:
# dataframe for fourth csv file 

# Underlying Cause of Death, 1999-2019.csv
file_path4 = '../data-mining-semester-project/data/Underlying Cause of Death, 1999-2019.csv'

# Load the data into a DataFrame 
dfCDeath = pd.read_csv(file_path4)
dfCDeath.head()
dfCDeath.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   County                                           4999 non-null   object 
 1   County Code                                      4999 non-null   object 
 2   Drug/Alcohol Induced                             4998 non-null   object 
 3   Drug/Alcohol Induced Code                        4998 non-null   object 
 4   Drug/Alcohol Induced Cause                       4998 non-null   object 
 5   Drug/Alcohol Induced Cause Code                  4998 non-null   object 
 6   Deaths                                           4998 non-null   float64
 7   Population                                       4998 non-null   float64
 8   Crude Rate                                       4998 non-null   object 
 9   Crude Rate Lower 95% Confidenc

In [43]:
# state death data covid19
file_path5 = '../data-mining-semester-project/data/countyCOVIDdeath.csv'

dfStateC19Death = pd.read_csv(file_path5)
dfStateC19Death.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [47]:
#county death data covid19
file_path6 = '../data-mining-semester-project/data/countyCOVIDdeath.csv'
dfCountyC19Death = pd.read_csv(file_path6)
dfCountyC19Death.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [32]:
# death data for type of drug 
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path4 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts-3.csv'

# Load the data into a DataFrame 
dfTypeDrug = pd.read_csv(file_path3)
dfTypeDrug.dropna()
dfTypeDrug.head()

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
0,AK,2015,April,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
1,AK,2015,April,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
2,AK,2015,April,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
3,AK,2015,April,12 month-ending,Number of Deaths,"4,133",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
4,AK,2015,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
